<h1>SQL Server 2019 Big Data Cluster</h1>


## Try to connect to the SQL Server master instance

In [1]:
select @@version, @@servername

(1 row affected)

Total execution time: 00:00:00.182

(No column name),(No column name)
Microsoft SQL Server 2019 (RTM-CU3) (KB4538853) - 15.0.4023.6 (X64) Mar 4 2020 00:59:26 Copyright (C) 2019 Microsoft Corporation Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>,master-0


## Create a demo database

In [2]:
CREATE DATABASE DemoDB;
GO

Commands completed successfully.

Total execution time: 00:00:10.177

## and an external datasource

In [3]:
USE DemoDB;
GO
CREATE EXTERNAL DATA SOURCE [SqlStoragePool]
    WITH (LOCATION = N'sqlhdfs://controller-svc:8080/default');

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.281

## Viewing the file : 6 header rows to exclude
## So, creating a file format accordingly

In [4]:
USE DemoDB;
GO
CREATE EXTERNAL FILE FORMAT [FileFormat_WxLog]
    WITH (FORMAT_TYPE = DELIMITEDTEXT, 
          FORMAT_OPTIONS (FIELD_TERMINATOR = N';', 
                          STRING_DELIMITER = N'\"', 
                          FIRST_ROW = 7));

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.310

## and then create the external table with a structure reflecting CSV structure file

In [5]:
USE DemoDB;
GO
CREATE EXTERNAL TABLE [dbo].[WxLog]
(
    [Date] [varchar](50) NOT NULL,
	[Time] [varchar](50) NOT NULL,
	[Baro] [varchar](50) NULL,
	[QNH] [varchar](50) NULL,
	[Gust Speed] [varchar](50) NULL,
	[Gust Dir] [varchar](50) NULL,
	[Avg Speed] [varchar](50) NULL,
	[Avg Dir] [varchar](50) NULL,
	[Rain Rate] [varchar](50) NULL,
	[Rain] [varchar](50) NULL,
	[UV] [varchar](50) NULL,
	[Temp 0] [varchar](50) NULL,
	[DewPt 0] [varchar](50) NULL,
	[RH 0] [varchar](50) NULL,
	[Temp 1] [varchar](50) NULL,
	[DewPt 1] [varchar](50) NULL,
	[RH 1] [varchar](50) NULL,
	[Temp 2] [varchar](50) NULL,
	[DewPt 2] [varchar](50) NULL,
	[RH 2] [varchar](50) NULL
)
WITH (LOCATION = N'/meteo/WxLog.csv', 
      DATA_SOURCE = [SqlStoragePool], 
	  FILE_FORMAT = [FileFormat_WxLog]);

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.271

## Now we can run some testing

In [6]:
USE DemoDB;
GO
SELECT TOP 10 * FROM [dbo].[WxLog]

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:01.131

Date,Time,Baro,QNH,Gust Speed,Gust Dir,Avg Speed,Avg Dir,Rain Rate,Rain,UV,Temp 0,DewPt 0,RH 0,Temp 1,DewPt 1,RH 1,Temp 2,DewPt 2,RH 2
11/05/2013,13:30,1024.00,1024.00,14.04,293,8.71,268,0.00,366.5,NULL,21.40,8.66,44,17.30,-1.37,28,NULL,NULL,
11/05/2013,13:31,1024.00,1024.00,9.72,293,9.00,293,0.00,366.5,NULL,21.40,8.66,44,17.40,-1.29,28,19.50,19.50,100
11/05/2013,13:32,1024.00,1024.00,12.60,293,9.79,288,0.00,366.5,NULL,21.40,8.66,44,17.60,-0.63,29,19.60,19.60,100
11/05/2013,13:33,1024.00,1024.00,12.24,293,9.43,266,0.00,366.5,NULL,21.40,8.66,44,17.50,-1.20,28,19.60,19.60,100
11/05/2013,13:34,1024.00,1024.00,12.60,270,9.62,266,0.00,366.5,NULL,21.40,8.66,44,17.85,-0.42,29,19.50,19.50,100
11/05/2013,13:35,1024.00,1024.00,11.88,293,9.00,268,0.00,366.5,NULL,21.40,8.66,44,18.00,-0.29,29,19.50,19.50,100
11/05/2013,13:36,1024.00,1024.00,18.36,225,9.59,256,0.00,366.5,NULL,21.40,8.82,45,18.00,-0.77,28,19.50,19.50,100
11/05/2013,13:37,1024.00,1024.00,14.04,248,11.95,234,0.00,366.5,NULL,21.43,9.02,45,18.00,-0.77,28,19.50,19.50,100
11/05/2013,13:38,1024.00,1024.00,11.88,248,12.60,241,0.00,366.5,NULL,21.50,9.08,45,18.00,-1.27,27,19.40,19.40,100
11/05/2013,13:39,1024.00,1024.00,16.56,248,11.29,243,0.00,366.5,NULL,21.50,9.08,45,18.00,-1.27,27,19.40,19.40,100


# We can switch to Spark engine to query the HDFS storage
## with the PySpark kernel



In [ ]:
# list all SQL Server 2019 BDC pods using PowerShell
kubectl get pods --namespace mssql-cluster

In [2]:
# Read the CSV file(s) into a spark dataframe and print schema
results = spark.read \
    .option("inferSchema", "true") \
    .csv('/csvfiles/temperature-last-year_poolhouse.csv') \
    .toDF("DateTime","Humidity","Temperature","Temperature_range (low)","Temperature_range (high)")
results.printSchema()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1605909717761_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DateTime: timestamp (nullable = true)
 |-- Humidity: double (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Temperature_range (low): double (nullable = true)
 |-- Temperature_range (high): double (nullable = true)

In [3]:
results.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

366

In [3]:
results.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|2018-05-14 00:00:00|    80.0|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|    88.0|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|    83.0|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|    84.0|      14.69|                   12.9|                    18.1|
|2018-05-18 00:00:00|    82.0|      15.91|                   11.1|                    20.8|
|2018-05-19 00:00:00|    76.0|      17.69|                   13.9|                    21.6|
|2018-05-20 00:00:00|    67.0|      19.07|                   12.4|                    24.5|
|2018-05-21 00:00:00|    65.0|      19.26|                   14.3|              

In [5]:
results.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|2018-05-14 00:00:00|    80.0|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|    88.0|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|    83.0|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|    84.0|      14.69|                   12.9|                    18.1|
|2018-05-18 00:00:00|    82.0|      15.91|                   11.1|                    20.8|
+-------------------+--------+-----------+-----------------------+------------------------+
only showing top 5 rows

## We can filter the dataframe

In [5]:
results.filter("Humidity > 70").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|2018-05-14 00:00:00|    80.0|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|    88.0|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|    83.0|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|    84.0|      14.69|                   12.9|                    18.1|
|2018-05-18 00:00:00|    82.0|      15.91|                   11.1|                    20.8|
|2018-05-19 00:00:00|    76.0|      17.69|                   13.9|                    21.6|
|2018-05-28 00:00:00|    82.0|      18.27|                   16.4|                    19.8|
|2018-05-29 00:00:00|    82.0|       19.0|                   16.4|              

In [7]:
results.filter("Humidity > 70").filter("Temperature > 15").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|2018-05-18 00:00:00|    82.0|      15.91|                   11.1|                    20.8|
|2018-05-19 00:00:00|    76.0|      17.69|                   13.9|                    21.6|
|2018-05-28 00:00:00|    82.0|      18.27|                   16.4|                    19.8|
|2018-05-29 00:00:00|    82.0|       19.0|                   16.4|                    22.2|
|2018-05-30 00:00:00|    84.0|      18.23|                   16.2|                    20.6|
|2018-05-31 00:00:00|    79.0|      18.97|                   15.7|                    23.4|
|2018-06-03 00:00:00|    73.0|      20.36|                   15.9|                    25.2|
|2018-06-04 00:00:00|    73.0|      20.77|                   15.5|              

In [4]:
results.select("temperature","Humidity").summary().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+
|summary|       temperature|          Humidity|
+-------+------------------+------------------+
|  count|               366|               366|
|   mean|15.283779680952737| 66.59827489034282|
| stddev|7.2870462623946715|13.000718440852483|
|    min|             -0.09|  36.9686098654709|
|    25%|              9.49|              57.0|
|    50%|             14.17|              67.0|
|    75%|             21.64|              78.0|
|    max|             30.28|              94.0|
+-------+------------------+------------------+

## We can switch a TSQL like syntax to query the dataframe

In [7]:
results.select("temperature","Humidity").show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+
|temperature|Humidity|
+-----------+--------+
|      10.06|    80.0|
|      11.83|    88.0|
|      13.47|    83.0|
|      14.69|    84.0|
|      15.91|    82.0|
|      17.69|    76.0|
|      19.07|    67.0|
|      19.26|    65.0|
|      19.31|    69.0|
|      20.69|    63.0|
+-----------+--------+
only showing top 10 rows

# We can also use some real TSQL statements.
Let's creae a kind of view ands make some queries

In [5]:
results.createOrReplaceTempView("poolhouse")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark.sql("SELECT * from poolhouse").show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|2018-05-14 00:00:00|    80.0|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|    88.0|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|    83.0|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|    84.0|      14.69|                   12.9|                    18.1|
|2018-05-18 00:00:00|    82.0|      15.91|                   11.1|                    20.8|
|2018-05-19 00:00:00|    76.0|      17.69|                   13.9|                    21.6|
|2018-05-20 00:00:00|    67.0|      19.07|                   12.4|                    24.5|
|2018-05-21 00:00:00|    65.0|      19.26|                   14.3|              

In [10]:
spark.sql("SELECT * from poolhouse LIMIT 10").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|2018-05-14 00:00:00|    80.0|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|    88.0|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|    83.0|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|    84.0|      14.69|                   12.9|                    18.1|
|2018-05-18 00:00:00|    82.0|      15.91|                   11.1|                    20.8|
|2018-05-19 00:00:00|    76.0|      17.69|                   13.9|                    21.6|
|2018-05-20 00:00:00|    67.0|      19.07|                   12.4|                    24.5|
|2018-05-21 00:00:00|    65.0|      19.26|                   14.3|              

In [11]:
spark.sql("SELECT MIN(Temperature),MAX(Temperature),AVG(Temperature) from poolhouse").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------------+------------------+
|min(Temperature)|max(Temperature)|  avg(Temperature)|
+----------------+----------------+------------------+
|           -0.09|           30.28|15.283779680952737|
+----------------+----------------+------------------+

In [12]:
spark.sql("SELECT DateTime,Temperature,LEAD(Temperature) OVER (order by DateTime) as NextValue,avg(Temperature) OVER () as avgTemp from poolhouse").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+---------+------------------+
|           DateTime|Temperature|NextValue|           avgTemp|
+-------------------+-----------+---------+------------------+
|2018-05-14 00:00:00|      10.06|    11.83|15.283779680952737|
|2018-05-15 00:00:00|      11.83|    13.47|15.283779680952737|
|2018-05-16 00:00:00|      13.47|    14.69|15.283779680952737|
|2018-05-17 00:00:00|      14.69|    15.91|15.283779680952737|
|2018-05-18 00:00:00|      15.91|    17.69|15.283779680952737|
|2018-05-19 00:00:00|      17.69|    19.07|15.283779680952737|
|2018-05-20 00:00:00|      19.07|    19.26|15.283779680952737|
|2018-05-21 00:00:00|      19.26|    19.31|15.283779680952737|
|2018-05-22 00:00:00|      19.31|    20.69|15.283779680952737|
|2018-05-23 00:00:00|      20.69|    21.14|15.283779680952737|
|2018-05-24 00:00:00|      21.14|    20.15|15.283779680952737|
|2018-05-25 00:00:00|      20.15|    21.54|15.283779680952737|
|2018-05-26 00:00:00|      21.54|    21.87|15.283779680

## We can also work on multiple files in the same folder

In [13]:
allfiles = spark.read \
    .option("inferSchema", "true") \
    .csv('/csvfiles/*.csv') \
    .toDF("DateTime","Humidity","Temperature","Temperature_range (low)","Temperature_range (high)")
allfiles.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2196

In [14]:
allfiles.select("temperature","Humidity").summary().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+------------------+
|summary|      temperature|          Humidity|
+-------+-----------------+------------------+
|  count|             2196|              2196|
|   mean|20.37647325436153|56.981552169311236|
| stddev|5.735156011715953|12.564311175829408|
|    min|            -0.09|              33.0|
|    25%|            20.25|              47.0|
|    50%|            22.08|              55.0|
|    75%|            23.47|              65.0|
|    max|            30.28|              94.0|
+-------+-----------------+------------------+

## It is also possible to use the JOIN operator between dataframes

In [15]:
bathroom = spark.read \
    .option("inferSchema", "true") \
    .csv('/csvfiles/temperature-last-year_salledebain.csv') \
    .toDF("DateTime","Humidity","Temperature","Temperature_range (low)","Temperature_range (high)")

livingroom = spark.read \
    .option("inferSchema", "true") \
    .csv('/csvfiles/temperature-last-year_sejour.csv') \
    .toDF("DateTime","Humidity","Temperature","Temperature_range (low)","Temperature_range (high)")    

bathroom.select("DateTime","temperature","Humidity").join(livingroom.select("DateTime","temperature","Humidity"),"DateTime").show(10)  


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+--------+-----------+--------+
|           DateTime|temperature|Humidity|temperature|Humidity|
+-------------------+-----------+--------+-----------+--------+
|2018-05-14 00:00:00|      21.32|    57.0|      21.78|    48.0|
|2018-05-15 00:00:00|      21.27|    58.0|      21.61|    50.0|
|2018-05-16 00:00:00|      21.15|    61.0|      21.73|    52.0|
|2018-05-17 00:00:00|      21.14|    64.0|      22.14|    55.0|
|2018-05-18 00:00:00|      21.63|    67.0|      22.53|    56.0|
|2018-05-19 00:00:00|      21.83|    68.0|      22.39|    59.0|
|2018-05-20 00:00:00|      21.78|    64.0|      22.84|    57.0|
|2018-05-21 00:00:00|       22.1|    63.0|      22.94|    57.0|
|2018-05-22 00:00:00|      22.55|    68.0|      23.25|    58.0|
|2018-05-23 00:00:00|      22.74|    68.0|      23.58|    57.0|
+-------------------+-----------+--------+-----------+--------+
only showing top 10 rows